### --- Day 17: Chronospatial Computer ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/17

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 17);
WriteLines(inputLines);

Loading puzzle file: Day17.txt
Total lines: 5
Max line length: 40

Register A: 38610541
Register B: 0
Register C: 0

Program: 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0


In [4]:
string[] testInputLines =
[
    "Register A: 729",
    "Register B: 0",
    "Register C: 0",
    "",
    "Program: 0,1,5,4,3,0",
];

In [5]:
using Register = ulong; // Need 64-bit numbers for Part 2
using Instr = byte; // Instructions are octal

delegate void Instruction(Instr operand);

In [6]:
class Processor
{
    public Processor()
    {
        PopulateInstructions();
    }
    
    // The computer also has three registers named A, B, and C, but these
    // registers aren't limited to 3 bits and can instead hold any integer.
    public Register RegisterA { get; private set; }
    public Register RegisterB { get; private set; }
    public Register RegisterC { get; private set; }

    // A number called the instruction pointer identifies the position in the
    // program from which the next opcode will be read; it starts at 0, pointing at
    // the first 3-bit number in the program.
    public int InstructionPointer { get; private set; }

    public void Process(Register a, Register b, Register c, Instr[] instructions)
    {
        (RegisterA, RegisterB, RegisterC) = (a, b, c);
        InstructionPointer = 0;
        _output = new();

        while (InstructionPointer < instructions.Length)
        {
            var (inst, operand) = instructions[InstructionPointer..(InstructionPointer+2)];
            var instruction = _instructionLookup[inst];
            instruction(operand);
        }
    }

    StringBuilder _output;
    public string Output => _output.ToString();

    // Operands
    /////

    Register Combo(Instr i) => i switch {
        0 => 0,
        1 => 1,
        2 => 2,
        3 => 3,
        4 => RegisterA,
        5 => RegisterB,
        6 => RegisterC,
        _ => throw new ArgumentException($"Unexpected combo operand")
    };

    // Instructions
    /////

    Dictionary<Instr, Instruction> _instructionLookup;

    void PopulateInstructions()
    {
        _instructionLookup = new()
        {
            { 0, Adv_0 },
            { 1, Bxl_1 },
            { 2, Bst_2 },
            { 3, Jnz_3 },
            { 4, Bxc_4 },
            { 5, Out_5 },
            { 6, Bdv_6 },
            { 7, Cdv_7 }
        };
    }

    // The eight instructions are as follows:

    // The adv instruction (opcode 0) performs division. The numerator is the
    // value in the A register. The denominator is found by raising 2 to the power
    // of the instruction's combo operand. (So, an operand of 2 would divide A by 4
    // (2^2); an operand of 5 would divide A by 2^B.) The result of the division
    // operation is truncated to an integer and then written to the A register.
    void Adv_0(Instr operand) {
        RegisterA = RegisterA / (Register)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }

    // The bxl instruction (opcode 1) calculates the bitwise XOR of register B
    // and the instruction's literal operand, then stores the result in register B.
    void Bxl_1(Instr operand) {
        RegisterB = RegisterB ^ (Register)operand;
        InstructionPointer += 2;
    }

    // The bst instruction (opcode 2) calculates the value of its combo operand
    // modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to
    // the B register.
    void Bst_2(Instr operand) {
        RegisterB = Combo(operand) % 8;
        InstructionPointer += 2;
    }

    // The jnz instruction (opcode 3) does nothing if the A register is 0.
    // However, if the A register is not zero, it jumps by setting the instruction
    // pointer to the value of its literal operand; if this instruction jumps, the
    // instruction pointer is not increased by 2 after this instruction.
    void Jnz_3(Instr operand) {
        if (RegisterA is 0) {
            InstructionPointer += 2;
            return; 
        }
        InstructionPointer = operand;
    }

    // The bxc instruction (opcode 4) calculates the bitwise XOR of register B
    // and register C, then stores the result in register B. (For legacy reasons,
    // this instruction reads an operand but ignores it.)
    void Bxc_4(Instr operand) {
        RegisterB = RegisterB ^ RegisterC;
        InstructionPointer += 2;
    }

    // The out instruction (opcode 5) calculates the value of its combo operand
    // modulo 8, then outputs that value. (If a program outputs multiple values,
    // they are separated by commas.)
    void Out_5(Instr operand) {
        if (_output.Length > 0) {
            _output.Append(",");
        }
        _output.Append(Combo(operand) % 8);
        InstructionPointer += 2;
    }

    // The bdv instruction (opcode 6) works exactly like the adv instruction
    // except that the result is stored in the B register. (The numerator is still
    // read from the A register.)
    void Bdv_6(Instr operand) {
        RegisterB = RegisterA / (Register)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }

    // The cdv instruction (opcode 7) works exactly like the adv instruction
    // except that the result is stored in the C register. (The numerator is still
    // read from the A register.)
    void Cdv_7(Instr operand) {
        RegisterC = RegisterA / (Register)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }
}

In [7]:
string RunProcessor(string[] inputLines)
{
    var numbers = inputLines.ParseAll(@"\d+").ToArray();

    var (a, b, c) = numbers.Select(Register.Parse).ToArray();
    var instructions = numbers[3..].Select(Instr.Parse).ToArray();
    
    Processor processor = new();
    processor.Process(a, b, c, instructions);
    return processor.Output;
}

In [8]:
// After the above program halts, its final output will be 4,6,3,5,6,3,5,2,1,0.

var testAnswer = RunProcessor(testInputLines);
Console.WriteLine(testAnswer);

4,6,3,5,6,3,5,2,1,0


In [9]:
// Using the information provided by the debugger, initialize the registers to
// the given values, then run the program. Once it halts, what do you get if you
// use commas to join the values it output into a single string?

var part1Answer = RunProcessor(inputLines);
Console.WriteLine(part1Answer);

7,5,4,3,4,5,3,4,6


In [10]:
// 7,5,4,3,4,5,3,4,6 is correct!
Ensure("7,5,4,3,4,5,3,4,6", part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/17

This was a tricky one! A few hard-fought observations and a hint or two eventually led me to the answer.

Using this basic function below to observe the output...

In [12]:
Instr[] testInstructions = [0,3,5,4,3,0];
Instr[] realInstructions = [2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0];

string ProcessOne(Register registerA, Instr[] instructions)
{
    var processor = new Processor();
    processor.Process(registerA, 0, 0, instructions);
    return processor.Output;
}

foreach (Register i in Enumerable.Range(0, 20))
{
    Console.WriteLine(ProcessOne(i, testInstructions));
}

0
0
0
0
0
0
0
0
1,0
1,0
1,0
1,0
1,0
1,0
1,0
1,0
2,0
2,0
2,0
2,0


...we notice with the sample instructions that the output is an octal counter!

#### Hypothesis 1: The instructions are a counter with the digits offset

Result: failed. The digits output are octal, but the digits do not cycle like a counter would. Sometimes the same digit is repeated, eg, `64` gives `404`, but so does `65`

In [13]:
foreach (Register i in Enumerable.Range(54, 20))
{
    Console.WriteLine(ProcessOne(i, realInstructions));
}

2,2
3,2
0,3
4,3
1,3
1,3
1,3
2,3
2,3
3,3
4,0,4
4,0,4
6,0,4
7,0,4
2,0,4
5,0,4
2,0,4
2,0,4
0,4,4
4,4,4


#### Hypothesis 2: Subsets of the instructions repeat every `N` cycles

Eg, the end result for `3,3,0` comes around every `N` iterations, then we can jump by `N` iterations until we find the next subset `0,3,3,0`, then increase again until we find the next subset, and so on.

Result: failed. The test shows no obvious cyclic repetition.

In [14]:
void FindCycles(string endsWith, Register max = 2_000_000)
{
    Register last = 0;
    for (Register i = 0; i < max; i++)
    {
        var latest = ProcessOne(i, realInstructions);
        if (latest.EndsWith(endsWith))
        {
            Console.WriteLine($"Iteration {i} ends with {endsWith}: {latest} (+{i - last})");
            last = i;
        }
    }
}

// Program: 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
FindCycles("5,5,0,3,3,0");

Iteration 152996 ends with 5,5,0,3,3,0: 5,5,0,3,3,0 (+152996)
Iteration 152999 ends with 5,5,0,3,3,0: 5,5,0,3,3,0 (+3)
Iteration 154155 ends with 5,5,0,3,3,0: 5,5,0,3,3,0 (+1156)
Iteration 1223968 ends with 5,5,0,3,3,0: 4,5,5,0,3,3,0 (+1069813)
Iteration 1223969 ends with 5,5,0,3,3,0: 4,5,5,0,3,3,0 (+1)
Iteration 1223970 ends with 5,5,0,3,3,0: 2,5,5,0,3,3,0 (+1)
Iteration 1223971 ends with 5,5,0,3,3,0: 7,5,5,0,3,3,0 (+1)
Iteration 1223972 ends with 5,5,0,3,3,0: 1,5,5,0,3,3,0 (+1)
Iteration 1223973 ends with 5,5,0,3,3,0: 3,5,5,0,3,3,0 (+1)
Iteration 1223974 ends with 5,5,0,3,3,0: 0,5,5,0,3,3,0 (+1)
Iteration 1223975 ends with 5,5,0,3,3,0: 7,5,5,0,3,3,0 (+1)
Iteration 1223992 ends with 5,5,0,3,3,0: 0,5,5,0,3,3,0 (+17)
Iteration 1223993 ends with 5,5,0,3,3,0: 4,5,5,0,3,3,0 (+1)
Iteration 1223994 ends with 5,5,0,3,3,0: 1,5,5,0,3,3,0 (+1)
Iteration 1223995 ends with 5,5,0,3,3,0: 1,5,5,0,3,3,0 (+1)
Iteration 1223996 ends with 5,5,0,3,3,0: 1,5,5,0,3,3,0 (+1)
Iteration 1223997 ends with 5,5,0,

At this point it was time to [check Reddit for some hints](https://www.reddit.com/r/adventofcode/comments/1hgcuw8/2024_day_17_part_2_any_hints_folks/).

The hint that got me going in the right direction was to think of the input register value as an octal number. When doing this, we note that the number of octal digits of the input register matches the number of digits of the output. So we know our register value is going to be a 16-digit number, in octal. (16 digits x 4 bits = 64 bit number).

With a bit more exploration, we notice that as we change digits `1,2,...n` from the start of our input, the corresponding digits `1,2,...n` from the _end_ of the output correspondingly change. Sometimes the output digit prior to the `n`th digit will also change, but that doesn't matter as we are not looking at that part of the output yet.

To put it another way: the digits `1-n` at the start of the input will generate consistent values for the values `1-n` at the end of the output, even as digits `n+1`, `n+2`, etc are subsequently mutated.

Eg:

```
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,4
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,7
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0
...
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,0,4,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,2,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,0,7,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,4,1,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,0,3,0
...
4,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,0,4,3,0
4,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,0
4,5,3,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,0,5,3,0
4,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,1,3,0
4,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,0,3,3,0
```

This is the key insight that allows us to find the solution. We can increment each input digit `i`, check the `-i`th digit of the output and see if it matches our expected, and if so, move to the next digit until the target string is reached.

This technique proves to be successful! Although the final trick is that upon finding a match for digit `i`, you cannot immediately move on to digit `i+1`- there might be multiple matches, eg:

```
Checking for 3,3,0:

4,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,0
...and
4,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0 => 4,4,4,4,4,4,4,4,4,4,4,4,0,3,3,0
```

When taking only the first match on every `i`th digit, the very last digit fails to match. I guess the earlier input digits affect later digits in a way that probably makes sense if I were to study the logical flow of the input instructions. Nonetheless, searching through all matching inputs yields the correct solution.

In [15]:
/// <summary>Convert from Octal -> Decimal</summary>
Register FromOctal(params Instr[] octalDigits)
{
    checked
    {
        Register result = 0;
        Register exp = 1;
        for (var i = 0; i < octalDigits.Length; i++)
        {
            var digit = octalDigits[^(i + 1)];
            result += digit * exp;

            exp *= 8;
        }

        return result;
    }
}

In [16]:
// Use this to represent a match that we need to search: assign values 0-7 to
// digit i in checkDigits, see if we now have a match for digit -i in the output
using CheckIndex = (Instr[] checkDigits, int i);

IEnumerable<Register> FindMatches(string targetString)
{
    var instructions = targetString.Split(',').Select(Instr.Parse).ToArray();
    
    Queue<CheckIndex> queue = new();
    var targetLength = targetString.Split(',').Count();
    queue.Enqueue((new Instr[targetLength], 0));

    while (queue.TryDequeue(out var nextCheck))
    {
        var (checkDigits, i) = nextCheck;
        var targetSubstring = targetString.Substring(targetString.Length - (2 * i + 1));

        foreach (Instr j in Enumerable.Range(0, 8))
        {
            checkDigits[i] = j;

            var checkString = ProcessOne(FromOctal(checkDigits), instructions);

            if (checkString.EndsWith(targetSubstring))
            {
                // Found a substring
                if (checkString == targetString)
                {
                    // Found the whole target string!
                    yield return FromOctal(checkDigits);
                    continue;
                }
                queue.Enqueue((checkDigits.ToArray(), i + 1));
            }
        }
    }
}

var matches =  FindMatches("2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0");
foreach (var match in matches)
{
    Console.WriteLine($"Found {match} -> {ProcessOne(match, realInstructions)}");
}

Found 164278899142333 -> 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
Found 164278899142589 -> 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0


In [17]:
Register FindLowestMatch(string targetString) => FindMatches(targetString).Min();

// What is the lowest positive initial value for register A that causes the program to output a copy of itself?

var part2Answer = FindLowestMatch("2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0");
Console.WriteLine(part2Answer);

164278899142333


In [18]:
// 164278899142333 is correct!
Ensure(164278899142333u, part2Answer);